# This code does face swapping without skin color of source videos changed 



## How to use this code:
1. Use `cd` command to change to the location where you store the test videos
2. Change `source_name` and `target_name` to the names original videos you want to use as source and target.
3. Change `source_path` and `target_path` to folders where you want to save the extracted source images and target images from the original videos.
4. Change `predictor_path` to where you store the landmark detection training dataset.
5. Change `result_img_path` to where you want to store the resulting images.
6. Change `result_video_path` to where you want the output videos be.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/videos

/content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/videos


# Define all the folder paths

In [ ]:
source_path = '/content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/Source' # the folder where the source image you want to save to
target_path = '/content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/Target' # the folder where the target image you want to save to
source_name = 'MrRobot.mp4' # name of source video
target_name = 'FrankUnderwood.mp4' # name of target video

In [ ]:
predictor_path = '/content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/shape_predictor_68_face_landmarks.dat' # path of the face predictor
result_img_path = '/content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/result_img' # save the image result after warping
result_video_path = '/content/drive/Shareddrives/CIS581 Final Project/Notebook/BoWu/result_video' # save the video after putting images together

# Get the image from source and target videos

In [ ]:
import cv2
 
# Opens the inbuilt camera of laptop to capture video.
source_cap = cv2.VideoCapture(source_name)
target_cap = cv2.VideoCapture(target_name)
# Get FPS of current video
current_FPS = source_cap.get(cv2.CAP_PROP_FPS)
print('Current FPS of video:', current_FPS)
# Set Target FPS you want, must 30, 60, or 120
target_FPS = current_FPS ### Change this
fold = current_FPS//target_FPS
i = 0
count = 0

while(source_cap.isOpened()):
    ret, frame = source_cap.read()
    # This condition prevents from infinite looping
    # incase video ends.
    if ret == False:
        break
    # Save Frame by Frame into disk using imwrite method
    if fold == 0:
      cv2.imwrite(source_path + '/Frame'+ str(i) +'.jpg', frame)
      count += 1
    else:
      if i%fold==0:
        cv2.imwrite(source_path +'/Frame'+ str(int(i/fold)) +'.jpg', frame)
        count += 1
    i += 1
print("total output frames of source video:", count)
source_cap.release()
cv2.destroyAllWindows()

source_count = count

i = 0
count = 0

while(target_cap.isOpened()):
    ret, frame = target_cap.read()
    # This condition prevents from infinite looping
    # incase video ends.
    if ret == False:
        break
    # Save Frame by Frame into disk using imwrite method
    if fold == 0:
      cv2.imwrite(target_path + '/Frame'+ str(i) +'.jpg', frame)
      count += 1
    else:
      if i%fold==0:
        cv2.imwrite(target_path +'/Frame'+ str(int(i/fold)) +'.jpg', frame)
        count += 1
    i += 1
print("total output frames of target video:", count)
target_cap.release()
cv2.destroyAllWindows()
target_count = count

Current FPS of video: 25.0
total output frames of source video: 238
total output frames of target video: 223


# Landmark Detection

In [ ]:
def landmarkDetection(image, predictor_path):
  '''
  INPUT:
  image = cv2.imread('/PATH/Image.jpg')

  OUTPUT:
  image: Original image with 68 landmarks plotted
  shape (68,2): All landmark points
  '''

  # initialize dlib's face detector (HOG-based) and then create
  # the facial landmark predictor
  p = "shape_predictor_68_face_landmarks.dat"
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor(predictor_path)

  # Convert image to grayscale for landmark detection
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # detect faces in the grayscale image
  rects = detector(gray, 0)

  # loop over the face detections
  for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
        
    # loop over the (x, y)-coordinates for the facial landmarks
    # and draw them on the image
    # for (x, y) in shape:
    #   cv2.circle(image, (x, y), 2, (0, 255, 0), 2) # cv2.circle(image, center_coordinates, radius, color, thickness)
      
    # show the output image with the face detections + facial landmarks
    # result = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    return shape

# Define heleper functions needed for triangulation and swapping

In [ ]:
from imutils import face_utils
import numpy as np
import dlib
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

# Read points from text file
def readPoints(path):
    # Create an array of points.
    points = [];
    
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    

    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Check if a point is inside a rectangle
def rectContains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[0] + rect[2] :
        return False
    elif point[1] > rect[1] + rect[3] :
        return False
    return True


#calculate delanauy triangle
def calculateDelaunayTriangles(rect, points):
    #create subdiv
    subdiv = cv2.Subdiv2D(rect);
    
    # Insert points into subdiv
    for p in points:
        subdiv.insert(p)
    
    triangleList = subdiv.getTriangleList();

    print(subdiv)


    delaunayTri = []
    
    pt = []    
        
    for t in triangleList:        
        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])        
        
        if rectContains(rect, pt1) and rectContains(rect, pt2) and rectContains(rect, pt3):
            ind = []
            #Get face-points (from 68 face detector) by coordinates
            for j in range(0, 3):
                for k in range(0, len(points)):                    
                    if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
                        ind.append(k)    
            # Three points form a triangle. Triangle array corresponds to the file tri.txt in FaceMorph 
            if len(ind) == 3:                                                
                delaunayTri.append((ind[0], ind[1], ind[2]))
        
        pt = []        
            
    
    return delaunayTri
        

# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2) :
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = [] 
    t2Rect = []
    t2RectInt = []

    for i in range(0, 3):
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
        t2RectInt.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1.0, 1.0, 1.0), 16, 0);
    
    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    #img2Rect = np.zeros((r2[3], r2[2]), dtype = img1Rect.dtype)
    
    size = (r2[2], r2[3])

    img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)
    
    img2Rect = img2Rect * mask

    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
     
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect 
        

# The swapping function for img1 and 2

In [ ]:
def mainTran(img1, img2, ith, old_hull2, old_img2):
    img1Warped = np.copy(img2);    
    
    points1 = landmarkDetection(img1, predictor_path)
    points2 = landmarkDetection(img2, predictor_path)
    
    # Find convex hull
    hull1 = []
    hull2 = []

    original_hull_index = cv2.convexHull(np.array(points1), returnPoints = False) ### remember to change this points2
    # print(hullIndex)
    mouth_points = [
    # [48],  # <outer mouth>
    # [49],
    # [50],
    # [51],
    # [52],
    # [53],
    # [54],
    # [55],
    # [56],
    # [57],
    # [58],  # </outer mouth>
    [60],  # <inner mouth>
    [61],
    [62],
    [63],
    [64],
    [65],
    [66],
    [67],  # </inner mouth>
    ]
    hull_index = np.concatenate((original_hull_index, mouth_points))
          
    for i in range(0, len(hullIndex)):
        hull1.append(points1[int(hullIndex[i])])
        hull2.append(points2[int(hullIndex[i])])

    original_hull2 = []
    for i in range(0, len(original_hull_index)):
      original_hull2.append(points2[int(original_hull_index[i])])
    
  ########### Apply Optical Flow
    old_hull2 = np.array(old_hull2, np.float32)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img2_gray_prev = cv2.cvtColor(old_img2, cv2.COLOR_BGR2GRAY)

    hull2_next, *_ = cv2.calcOpticalFlowPyrLK(
            img2_gray_prev,
            img2_gray,
            old_hull2,
            np.array(hull2, np.float32),
            winSize=(101, 101),
            maxLevel=5,
            criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.001),
            )
    for i, _ in enumerate(hull2):
        if i < len(hull2_next):
          hull2[i] = 0.3 * np.array(hull2[i]) + 0.7 * hull2_next[i]


    
  ###########

    # Find delanauy traingulation for convex hull points
    sizeImg2 = img2.shape    
    rect = (0, 0, sizeImg2[1], sizeImg2[0])
     
    # dt = calculateDelaunayTriangles(rect, hull2)
    dt = Delaunay(np.array(hull1))

    dt = dt.simplices
    if len(dt) == 0:
        quit()
    
    # Apply affine transformation to Delaunay triangles
    for i in range(0, len(dt)):
        t1 = []
        t2 = []
        
        #get points for img1, img2 corresponding to the triangles
        for j in range(0, 3):
            t1.append(hull1[dt[i][j]])
            t2.append(hull2[dt[i][j]])
        
        warpTriangle(img1, img1Warped, t1, t2)
    
            
    # Calculate Mask
    hull8U = []
    for i in range(0, len(original_hull2)):
        hull8U.append((original_hull2[i][0], original_hull2[i][1]))
    
    mask = np.zeros(img2.shape, dtype = img2.dtype)  
    
    cv2.fillConvexPoly(mask, np.int32(hull8U), (255, 255, 255))
    
    r = cv2.boundingRect(np.float32([original_hull2]))    
    
    center = ((r[0]+int(r[2]/2), r[1]+int(r[3]/2)))
    
    # Clone seamlessly.
    output = cv2.seamlessClone(np.uint8(img1Warped), img2, mask, center, cv2.NORMAL_CLONE)
    # output = correct_colours(output, img2, points2)
    # result = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
    cv2.imwrite(result_img_path + '/' + str(ith) + '.jpg', output)

    old_hull2 = hull2
    old_img2 = img2
    return old_hull2, old_img2

# Main function you need to run for the image swapping

# Initialize the old_hull

In [ ]:
old_img2 = cv2.imread(target_path + '/Frame' + str(1) + '.jpg')
old_hull2 = []

points2 = landmarkDetection(old_img2, predictor_path)

hullIndex = cv2.convexHull(np.array(points2), returnPoints = False)
    # print(hullIndex)
          
for i in range(0, len(hullIndex)):
    old_hull2.append(points2[int(hullIndex[i])])
old_hull2 = np.array(old_hull2, np.float32)

In [ ]:
n_Image = 2 #min(source_count, target_count) # this is the total number of images you want use to reconstruct the video

for i in range(1, n_Image):
    target_img = cv2.imread(target_path + '/Frame' + str(i) + '.jpg')
    source_img = cv2.imread(source_path + '/Frame' + str(i) + '.jpg')
    
    old_hull2, old_img2 = mainTran(source_img, target_img, i, old_hull2, old_img2)

# Video reconstruction from images

In [ ]:
import cv2
import numpy as np
import glob


img_array = []

for filename in glob.glob(result_img_path+'/*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter(result_video_path + '/project_of.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()